# 이력서 분석 에이전트

오늘날 빠르게 진화하는 AI 환경에서 효과적으로 협업하는 정교한 에이전트 시스템을 구축하는 것은 여전히 중요한 과제입니다. LangChain 팀은 두 가지 강력한 새로운 Python 라이브러리인 `langgraph-supervisor` 와 `langgraph-swarm`을 출시하여 이러한 요구를 충족했습니다 . 이 글에서는 langgraph-supervisor를 통해 개발자가 계층적 구조를 갖춘 복잡한 다중 에이전트 시스템을 구축하는 방법을 살펴봅니다.

[샘플이력서 다운로드](https://www.resumeviking.com/templates/)

## 1. 환경 설정

In [14]:
%pip install -qU langgraph-supervisor langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [24]:
# 환경 변수 확인
from dotenv import load_dotenv
import os
load_dotenv(override=True)

# TAVILY_API_KEY= os.environ.get("TAVILY_API_KEY")
# print(TAVILY_API_KEY[:20])

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")
_set_env("OPENAI_API_KEY")

## 2. LLM 객체 생성

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini")

## 3. 에이전트 생성

In [32]:
%pip install -qU PyPDF2 langchain langgraph googlesearch-python python-docx langgraph-supervisor

Note: you may need to restart the kernel to use updated packages.


In [33]:
from langgraph.prebuilt import create_react_agent
from PyPDF2 import PdfReader
from docx import Document
from googlesearch import search

In [35]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    PDF 파일로부터 텍스트를 추출합니다.

    매개변수:
        pdf_path (str): 텍스트를 추출할 PDF 파일의 경로

    반환값:
        str: 추출된 텍스트 전체를 문자열로 반환
    """
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

def extract_text_from_docx(docx_path: str) -> str:
    """
    DOCX 파일로부터 텍스트를 추출합니다.

    매개변수:
        docx_path (str): 텍스트를 추출할 DOCX 파일의 경로

    반환값:
        str: 추출된 문단을 줄바꿈('\n')으로 구분하여 하나의 문자열로 반환
    """
    doc = Document(docx_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])

def resume_parser(resume_file_path: str):
    """
    이력서 파일(PDF 또는 DOCX)에서 텍스트를 추출합니다.

    매개변수:
        resume_file_path (str): 이력서 파일의 경로 (PDF 또는 DOCX 형식)

    반환값:
        str: 이력서에서 추출된 텍스트

    예외:
        ValueError: 지원하지 않는 파일 형식일 경우 발생
    """
    if resume_file_path.endswith(".pdf"):
        return extract_text_from_pdf(resume_file_path)
    elif resume_file_path.endswith(".docx"):
        return extract_text_from_docx(resume_file_path)
    else:
        raise ValueError("Unsupported file type")

# Create resume parser agent
resume_parser_agent = create_react_agent(
    llm,
    tools=[resume_parser],
    name="resume_parser_agent",
    prompt=(
        "당신은 이력서 파싱 전문가입니다. "
        "항상 resume_parser라는 하나의 도구만 사용하여 이력서를 파싱하세요."
    )
)

In [36]:
def general_question_answer(question: str):
    """
    일반적인 질문에 대한 답변을 생성합니다.

    매개변수:
        question (str): 사용자가 입력한 질문

    반환값:
        str: 모델이 생성한 답변 내용
    """
    response = model.invoke(question)
    return response.content

# Create general Q&A agent
general_question_answer_agent = create_react_agent(
    llm,
    tools=[general_question_answer],
    name="general_question_answer_agent",
    prompt=(
        "당신은 일반 질문에 답변하는 전문가입니다. "
        "항상 general_question_answer라는 하나의 도구만 사용하여 질문에 답하세요."
    )
)

In [37]:
def google_search(query: str):
    """
    주어진 검색어(query)를 사용하여 Google에서 검색을 수행하고, 상위 5개의 결과를 반환합니다.

    매개변수:
        query (str): 검색할 질의어 또는 키워드

    반환값:
        list: 검색 결과 URL 목록 (최대 5개)
    """
    return list(search(query, num_results=5))

# Create Google search agent
google_search_agent = create_react_agent(
    llm,
    tools=[google_search],
    name="google_search_agent",
    prompt=(
        "당신은 구글 검색 전문가입니다. "
        "항상 google_search라는 하나의 도구만 사용하여 인터넷을 검색하세요."
    )
)  

In [38]:
from langgraph_supervisor import create_supervisor

# Create supervisor workflow
workflow = create_supervisor(
    [resume_parser_agent, google_search_agent, general_question_answer_agent],
    model=llm,
    prompt=(
        "당신은 여러 에이전트를 관리하는 스마트한 팀 관리자입니다. 사용자 입력을 분석하고 적절한 에이전트에게 작업을 위임하세요:\n"
        "- 입력에 파일 경로나 '이력서(resume)'가 언급되어 있다면, resume_parser_agent를 사용하세요.\n"
        "- '검색(search)'이 포함되어 있거나 온라인에서 무언가를 찾으라는 요청이 있다면, google_search_agent를 사용하세요.\n"
        "- 그 외의 모든 질문이나 요청에 대해서는 general_question_answer_agent를 사용하세요.\n"
        "사용자의 입력을 기반으로 가장 적절한 에이전트를 선택하세요."
    ),
    output_mode="last_message"
)

In [39]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
app = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}

In [40]:
# Main interaction loop
while True:
    user_input = input("\nEnter your query (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    result = app.invoke({
        "messages": [{
            "role": "user",
            "content": user_input
        }]
    }, config=config)

    for m in result["messages"]:
        print(m.content)


Enter your query (or 'exit' to quit):  JULIE MONROE의 이력서에서 주소를 알려주ㅏ


Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.


JULIE MONROE의 이력서에서 주소를 알려주ㅏ

Successfully transferred to resume_parser_agent
이력서를 파싱할 수 있도록 해당 이력서 파일을 제공해 주시기 바랍니다. (PDF 또는 DOCX 형식)
Transferring back to supervisor
Successfully transferred back to supervisor
이력서에서 정보를 추출하려면 해당 이력서 파일을 제공해 주셔야 합니다. 파일을 업로드해주세요.



Enter your query (or 'exit' to quit):  D:\ws\2025-ksci-agent\resume


Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.


...


Enter your query (or 'exit' to quit):  2024 미국 대선 결과를 검색해서 알려줘


Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel is_last_step, ignoring it.
Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel remaining_steps, ignoring it.


JULIE MONROE의 이력서에서 주소를 알려주ㅏ

Successfully transferred to resume_parser_agent
이력서를 파싱할 수 있도록 해당 이력서 파일을 제공해 주시기 바랍니다. (PDF 또는 DOCX 형식)
Transferring back to supervisor
Successfully transferred back to supervisor
이력서에서 정보를 추출하려면 해당 이력서 파일을 제공해 주셔야 합니다. 파일을 업로드해주세요.
D:\ws\2025-ksci-agent\resume

Successfully transferred to resume_parser_agent
제공하신 경로의 파일 형식이 지원되지 않는 것 같습니다. 이력서 파일이 PDF 또는 DOCX 형식인지 확인해주시고, 해당 형식의 파일을 업로드해 주시기 바랍니다.
Transferring back to supervisor
Successfully transferred back to supervisor
이력서 파일을 PDF 또는 DOCX 형식으로 변환하거나 해당 형식으로 업로드해 주셔야 합니다. 지원하는 형식의 이력서를 제공해 주세요.
D:\ws\2025-ksci-agent\resume\New-York-Resume-Template-Creative.pdf

Successfully transferred to resume_parser_agent
JULIE MONROE의 이력서에서 추출된 주소는 다음과 같습니다:

**주소:**  
1515 Pacific Ave  
Los Angeles, CA 90291  
United States  

이외에 다른 정보가 필요하시면 말씀해 주세요!
Transferring back to supervisor
Successfully transferred back to supervisor
JULIE MONROE의 이력서에서 추출된 주소는 다음과 같습니다:

**주소:**  
1515 Pacific Ave  
Los Angeles, CA 9029


Enter your query (or 'exit' to quit):  exit


Goodbye!
